# 04 Derive our initial project metrics at 90 and 180 day intervals

Metrics:
- active_addresses_180D
- active_addresses_90D
- daily_active_addresses_180D
- daily_active_addresses_90D
- farcaster_users_180D
- farcaster_users_90D
- transactions_180D
- transactions_90D

In [1]:
import pandas as pd

In [2]:
projects = pd.read_csv('data/apps/project_apps_labeled.csv', index_col=0).set_index('uuid')
projects = projects[projects['project_type'] == 'App']
project_contract_types = projects['contract_type'].to_dict()

all_apps = pd.read_csv('data/apps/applications_reviewed.csv')
app_map = all_apps.set_index('uuid')['name'].to_dict()
all_apps['contract_type'] = all_apps['uuid'].map(project_contract_types)

all_apps.head(1)

,uuid,charmverseId,agoraProjectRefUID,id,recipient,time,name,status,profile_name,profile_url,...,chain_id,chain,flag_multiple_projects_same_profile,flag_creator_no_address,flag_app_missing_contract,flag_channel_no_channel,flag_charmverse_in_name,count_flags,has_flag,contract_type
0,89eb0a1c-36f6-4455-af98-c822064425bb,89eb0a1c-36f6-4455-af98-c822064425bb,0xea5bb5ac6f59ef70cb9bd8da5ba114b08055af4901f4...,0x66076854e0f9ce49078c76ee39e2e9fae61a8526f406...,0xE4EE538019673501F4B75de5aF5CC073Ec0A1487,1724860607,0x,approved,ewokafloka,https://warpcast.com/ewokafloka,...,8453.0,Base,False,False,False,False,False,0,0,dapp_contract


In [3]:
df = pd.read_parquet('data/raw_metric_data/project_events.parquet')
df = df[ df['uuid'].isin(projects.index)]

In [6]:
df.tail(1)

,uuid,chain,contract_address,contract_type,user_address,date,count_transactions,farcaster_id,farcaster_username,recipient
5521952,1c0ae0df-9238-497d-8fa0-ad05f381cf34,Base,0x52b7fdb72db7b1279919ec1cf69d3f5cb51d1243,trace_contract,0xd120c31eb8a5e43144361e3266d701b38ea4ed63,2024-08-28,6.0,None,None,0xf3B06b503652a5E075D423F97056DFde0C4b066F


In [7]:
projects.head(1)

,recipient,project_type,category,address,chain,contract_type
uuid,,,,,,
89eb0a1c-36f6-4455-af98-c822064425bb,0xE4EE538019673501F4B75de5aF5CC073Ec0A1487,App,DEX,0xdef1c0ded9bec7f1a1670819833240f027b25eff,Base,dapp_contract


## Part 1. Get metrics by UUID

In [8]:
metrics = []
errors = []
for uuid, app in projects.iterrows():
    
    name = app_map.get(uuid)
    
    if app['chain'] not in ['Optimism', 'Base', 'Frax', 'Mode', 'Metal', 'Zora']:
        errors.append({
            'uuid': uuid,
            'name': name,
            **app,
            'status': 'pending_chain_indexing'
        })
        continue
        
    if app['contract_type'] == 'factory_contract':
        errors.append({
            'uuid': uuid,
            'name': name,
            **app,
            'status': 'pending_factory_indexing'
        })
        continue
        
    if app['contract_type'] == 'unknown':
        errors.append({
            'uuid': uuid,
            'name': name,
            **app,
            'status': 'no_onchain_activity_180D'
        })
        continue

    
    dff_180D = df[df.uuid == uuid]
    dff_90D = dff_180D[dff_180D['date'] >= '2024-06-01']
    
    if len(dff_90D) < 1:
        errors.append({
            'uuid': uuid,
            'name': name,
            **app,
            'status': 'no_onchain_activity_90D'
        })
        continue
    
    daas_90 = dff_90D.groupby('date')['user_address'].nunique().sum() / 90
    daas_180 = dff_180D.groupby('date')['user_address'].nunique().sum() / 180
    
    project_metrics = {
        'uuid': uuid,
        'name': name,
        **app,
        'status': 'metrics_available',

        'transactions_90D': dff_90D['count_transactions'].sum(),
        'active_addresses_90D': dff_90D['user_address'].nunique(),
        'daily_active_addresses_90D': int(daas_90),
        'farcaster_users_90D': dff_90D['farcaster_id'].nunique(),
        
        'transactions_180D': dff_180D['count_transactions'].sum(),
        'active_addresses_180D': dff_180D['user_address'].nunique(),
        'daily_active_addresses_180D': int(daas_180),
        'farcaster_users_180D': dff_180D['farcaster_id'].nunique(),
    
    }
    metrics.append(project_metrics)

In [9]:
df_metrics = pd.concat(
    [pd.DataFrame(metrics), pd.DataFrame(errors)],
    axis=0,
    ignore_index=True
)
df_metrics

,uuid,name,recipient,project_type,category,address,chain,contract_type,status,transactions_90D,active_addresses_90D,daily_active_addresses_90D,farcaster_users_90D,transactions_180D,active_addresses_180D,daily_active_addresses_180D,farcaster_users_180D
0,89eb0a1c-36f6-4455-af98-c822064425bb,0x,0xE4EE538019673501F4B75de5aF5CC073Ec0A1487,App,DEX,0xdef1c0ded9bec7f1a1670819833240f027b25eff,Base,dapp_contract,metrics_available,1543115.0,154685.0,3984.0,9828.0,2811827.0,361243.0,5434.0,27040.0
1,b4d81636-0080-49bd-8039-0721b3ea716c,Aerodrome Finance,0xc687f7c9BDfF8d40cF4f789FA105f7dCF0771d6a,App,DEX,0x940181a94a35a4569e4529a3cdfb74e38fd98631,Base,token_contract,metrics_available,26251866.0,196067.0,7605.0,12760.0,36012251.0,300062.0,7534.0,21417.0
2,c31e36bc-69ad-40e8-a3b9-29f5c1d6d86a,AI,0xf1a871aF236eC0Da070C3b09302270Ccf4E35794,App,Real World Assets,0xf1a871af236ec0da070c3b09302270ccf4e35794,Base,dapp_contract,metrics_available,41.0,21.0,0.0,1.0,55.0,28.0,0.0,3.0
3,795a96c7-4d7e-4c58-8958-aad52d6a8f12,Air3,0x84EC34a8b64e9A40248599fe0f8afA9d66742Fe0,App,Art Marketplace,0xfd4ccd0ccab4e31cce600bd7f6b9a42623294b1c,Optimism,dapp_contract,metrics_available,1774.0,1104.0,13.0,261.0,1774.0,1104.0,6.0,261.0
4,657d3052-63b9-499a-8898-ee74a2d4c0e7,Amazing Hunt,0x9D63038e7f74bc089eb38e61Be8Da937918900D6,App,Art NFTs,0x0db19369d6d62237d065677cf05c1b0a1236bb0a,Optimism,dapp_contract,metrics_available,1179.0,3.0,0.0,0.0,1179.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,94685eed-3fee-462f-b141-862a016f363f,Testing testing,0xb1b9FFF08F3827875F91ddE929036a65f2A5d27d,App,DEX,0x90e1c94ca5e161acb420e59d8be5041db931c9dd,Base,unknown,no_onchain_activity_180D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231,ca0f8386-dd66-4486-8a50-060e2beb29aa,TLX,0x839d36595175E1F678110d52D8265a2FEa9607bA,App,DEX,0x4a1ee35f7a41b72c65ac429ec92c1221be78404c,Optimism,dapp_contract,no_onchain_activity_90D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
232,c798394d-fb13-4db6-b2a6-11dba0abae2f,UNI-Bridge,0xcDe21819d2bd07A44DE9365523FC870a31f755f1,App,Bridges,0xf9982e648ee8f9e3e9039b0071ba939c3bc19652,Optimism,unknown,no_onchain_activity_180D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
233,ceac6653-ea1d-43b8-9e02-8329e5ee2a90,Vault,0x3363b291a21cC692A5e07C9C63E3DF45F135EFcd,App,Payments,0x00000000000052068951aed201da868e29db48ac,Base,factory_contract,pending_factory_indexing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_metrics['chain'].value_counts()

chain
Base          135
Optimism       63
Mint            8
Mode            7
Zora            7
SwanChain       4
Kroma           4
Cyber           2
Orderly         2
Polynomial      2
Lisk            1
Name: count, dtype: int64

In [11]:
explorers = {
    'Base': "https://basescan.org/address/",
    'Optimism': "https://optimistic.etherscan.io/address/",
    'Zora': "https://zora.thesuperscan.io/address/",
    'Mode': "https://explorer.mode.network/address/",
    'Mint': "https://explorer.mintchain.io/address/",
    'SwanChain': "https://mainnet-explorer.swanchain.io/address/",
    'Kroma': "https://kromascan.com/address/",
    'Cyber': "https://cyberscan.co/address/",
    'Orderly': "https://explorer.orderly.network/address/",
    'Polynomial': "https://polynomialscan.io/address/",
    'Lisk': "https://blockscout.lisk.com/address/",
    'Redstone': "https://explorer.redstone.xyz/address/"
}
df_metrics['explorer'] = df_metrics.apply(lambda x: explorers.get(x['chain'])+x['address'], axis=1)
df_metrics

,uuid,name,recipient,project_type,category,address,chain,contract_type,status,transactions_90D,active_addresses_90D,daily_active_addresses_90D,farcaster_users_90D,transactions_180D,active_addresses_180D,daily_active_addresses_180D,farcaster_users_180D,explorer
0,89eb0a1c-36f6-4455-af98-c822064425bb,0x,0xE4EE538019673501F4B75de5aF5CC073Ec0A1487,App,DEX,0xdef1c0ded9bec7f1a1670819833240f027b25eff,Base,dapp_contract,metrics_available,1543115.0,154685.0,3984.0,9828.0,2811827.0,361243.0,5434.0,27040.0,https://basescan.org/address/0xdef1c0ded9bec7f...
1,b4d81636-0080-49bd-8039-0721b3ea716c,Aerodrome Finance,0xc687f7c9BDfF8d40cF4f789FA105f7dCF0771d6a,App,DEX,0x940181a94a35a4569e4529a3cdfb74e38fd98631,Base,token_contract,metrics_available,26251866.0,196067.0,7605.0,12760.0,36012251.0,300062.0,7534.0,21417.0,https://basescan.org/address/0x940181a94a35a45...
2,c31e36bc-69ad-40e8-a3b9-29f5c1d6d86a,AI,0xf1a871aF236eC0Da070C3b09302270Ccf4E35794,App,Real World Assets,0xf1a871af236ec0da070c3b09302270ccf4e35794,Base,dapp_contract,metrics_available,41.0,21.0,0.0,1.0,55.0,28.0,0.0,3.0,https://basescan.org/address/0xf1a871af236ec0d...
3,795a96c7-4d7e-4c58-8958-aad52d6a8f12,Air3,0x84EC34a8b64e9A40248599fe0f8afA9d66742Fe0,App,Art Marketplace,0xfd4ccd0ccab4e31cce600bd7f6b9a42623294b1c,Optimism,dapp_contract,metrics_available,1774.0,1104.0,13.0,261.0,1774.0,1104.0,6.0,261.0,https://optimistic.etherscan.io/address/0xfd4c...
4,657d3052-63b9-499a-8898-ee74a2d4c0e7,Amazing Hunt,0x9D63038e7f74bc089eb38e61Be8Da937918900D6,App,Art NFTs,0x0db19369d6d62237d065677cf05c1b0a1236bb0a,Optimism,dapp_contract,metrics_available,1179.0,3.0,0.0,0.0,1179.0,3.0,0.0,0.0,https://optimistic.etherscan.io/address/0x0db1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,94685eed-3fee-462f-b141-862a016f363f,Testing testing,0xb1b9FFF08F3827875F91ddE929036a65f2A5d27d,App,DEX,0x90e1c94ca5e161acb420e59d8be5041db931c9dd,Base,unknown,no_onchain_activity_180D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://basescan.org/address/0x90e1c94ca5e161a...
231,ca0f8386-dd66-4486-8a50-060e2beb29aa,TLX,0x839d36595175E1F678110d52D8265a2FEa9607bA,App,DEX,0x4a1ee35f7a41b72c65ac429ec92c1221be78404c,Optimism,dapp_contract,no_onchain_activity_90D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://optimistic.etherscan.io/address/0x4a1e...
232,c798394d-fb13-4db6-b2a6-11dba0abae2f,UNI-Bridge,0xcDe21819d2bd07A44DE9365523FC870a31f755f1,App,Bridges,0xf9982e648ee8f9e3e9039b0071ba939c3bc19652,Optimism,unknown,no_onchain_activity_180D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://optimistic.etherscan.io/address/0xf998...
233,ceac6653-ea1d-43b8-9e02-8329e5ee2a90,Vault,0x3363b291a21cC692A5e07C9C63E3DF45F135EFcd,App,Payments,0x00000000000052068951aed201da868e29db48ac,Base,factory_contract,pending_factory_indexing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://basescan.org/address/0x000000000000520...


In [12]:
cols = [
    'name', 'project_type', 'category', 'chain', 'explorer', 'contract_type', 'status',
    'transactions_90D', 'active_addresses_90D', 'daily_active_addresses_90D', 'farcaster_users_90D',
    'transactions_180D', 'active_addresses_180D', 'daily_active_addresses_180D', 'farcaster_users_180D',
    'uuid', 'recipient', 'address'
]
df_metrics[cols].to_csv('data/clean_metric_data/metrics_projects.csv')

## Part 2. Get metrics by recipient address

In [21]:
recipient_mapping = (
    all_apps
    .groupby(['recipient', 'name'])
    ['uuid']
    .unique()
    .apply(list)
    .to_dict()
)

In [28]:
recipient_metrics = []
recipient_errors = []
for (recipient,name),uuid_list in recipient_mapping.items():
    
    dff_180D = df[df.uuid.isin(uuid_list)]
    if len(dff_180D) < 1:
        continue
    
    dff_90D = dff_180D[dff_180D['date'] >= '2024-06-01']
    if len(dff_90D) < 1:
        recipient_errors.append({
            'recipient': recipient,
            'name': name,
            'uuid_list': uuid_list,
            'status': 'no_onchain_activity_90D'
        })
        continue
    
    daas_90 = dff_90D.groupby('date')['user_address'].nunique().sum() / 90
    daas_180 = dff_180D.groupby('date')['user_address'].nunique().sum() / 180
    
    project_metrics = {
        'recipient': recipient,
        'name': name,
        'status': 'metrics_available',

        'transactions_90D': dff_90D['count_transactions'].sum(),
        'active_addresses_90D': dff_90D['user_address'].nunique(),
        'daily_active_addresses_90D': int(daas_90),
        'farcaster_users_90D': dff_90D['farcaster_id'].nunique(),
        
        'transactions_180D': dff_180D['count_transactions'].sum(),
        'active_addresses_180D': dff_180D['user_address'].nunique(),
        'daily_active_addresses_180D': int(daas_180),
        'farcaster_users_180D': dff_180D['farcaster_id'].nunique(),
        
        'uuid_list': uuid_list
    }
    recipient_metrics.append(project_metrics)

In [29]:
df_recipient_metrics = pd.concat(
    [pd.DataFrame(recipient_metrics), pd.DataFrame(recipient_errors)],
    axis=0,
    ignore_index=True
)
df_recipient_metrics

,recipient,name,status,transactions_90D,active_addresses_90D,daily_active_addresses_90D,farcaster_users_90D,transactions_180D,active_addresses_180D,daily_active_addresses_180D,farcaster_users_180D,uuid_list
0,0x01133c3834a96aDd681a9C9Ad3b2f955968ed60F,ArbiLearn,metrics_available,36.0,10.0,0.0,0.0,36.0,10.0,0.0,0.0,"[6abc3227-4e7f-4786-863c-abdd23f488a2, 72d9170..."
1,0x013437412c41716f72B54fE66d7c11858aFe4894,BetBase,metrics_available,12901.0,2146.0,60.0,198.0,32785.0,4295.0,68.0,456.0,[8bd5539d-ef65-4f2c-9014-3fc11620f12f]
2,0x022541dF816F50CfC6ce60c9de80619F44f7CeF1,datalatte,metrics_available,73.0,4.0,0.0,3.0,73.0,4.0,0.0,3.0,[78f0eac0-c56e-489c-a4e3-9c700ccf4ce8]
3,0x02cdD55Bea4d8Ef7E7dDA6A60305a03dfaB0C128,/send,metrics_available,33636.0,1816.0,46.0,129.0,45966.0,3765.0,60.0,293.0,"[02873c40-7e3f-4f0c-b9ff-ba12466b834c, 3600073..."
4,0x035eBd096AFa6b98372494C7f08f3402324117D3,Namespace - ENS Subname Sorcery,metrics_available,1683.0,143.0,1.0,117.0,1683.0,143.0,0.0,117.0,[45ca2132-cb78-4e41-aca2-61f2fe8c71c1]
...,...,...,...,...,...,...,...,...,...,...,...,...
129,0x839d36595175E1F678110d52D8265a2FEa9607bA,TLX,no_onchain_activity_90D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ca0f8386-dd66-4486-8a50-060e2beb29aa]
130,0x9D63038e7f74bc089eb38e61Be8Da937918900D6,Bored Town,no_onchain_activity_90D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[0674ed13-3026-4dad-9565-b522a40c8eff]
131,0xAa66fC4433DFD254F05A720A8c8a2f1F2fbDCB58,baby Dragon,no_onchain_activity_90D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[9a65dbec-2fb8-4889-9ef3-e1e2bbef14d4]
132,0xeAb4717efb29f91429dB6322F42Edc53C94F0941,Avantis,no_onchain_activity_90D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[0592c036-8570-4c55-83bd-759ac99860ad]


In [30]:
df_recipient_metrics.to_csv('data/clean_metric_data/metrics_projects_by_recipient.csv')